NB! Несколько важных советов для этого задания дала Анна Николаева.

Take the data

In [309]:
import pandas as pd
import csv

In [310]:
train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE,encoding='utf-8').fillna(0)
len(train)

16406

In [4]:
#train.rename(index=str, columns={"class": "class_"}, inplace=True,) # inplace=True means the df is really changed
#train.head()

In [311]:
dev = pd.read_csv('dev.csv',sep = '\t', quoting=csv.QUOTE_NONE).fillna(0)

In [312]:
test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE).fillna(0)

In [16]:
# get all the sentences for our own model training
all_text = train['text'].tolist() + dev['text'].tolist() + test['text'].tolist()

#train_dev_labels1 = train['class_'].tolist() + dev['class_'].tolist()

# make a file with all sents
with open('./all_sents.txt', 'w') as f: 
    for sent in all_text:
     #   text = lemmatize_source(text)
        f.write(sent + '\n')

In [15]:
#all_text

In [313]:
# get train sentences in one text for normalizing
tr_text = "tyxz ".join(train['text'].tolist())
#len(tr_text)

In [314]:
# get dev sentences in one text for normalizing
dev_text = "tyxz ".join(dev['text'].tolist())
#len(dev_text)

In [315]:
# get test sentences in one text for normalizing
test_text = test['text'].tolist()
test_text_j = "\n".join(test_text)

In [316]:
# change postags in case we try an external model

import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)

{'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}


In [317]:
# get lemmas with postags

from pymystem3 import Mystem

punct = '!"#$%&\'()*+\./:;<=>?@[]^_`{|}~«»…“”*№,–-0123456789'
m = Mystem()
#MAX_length = 50

def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
  #  m = Mystem()
    processed = m.analyze(text)
    lem_tag_d = {}
    tagged = []
    count = 0
    
    for w in processed:
        count += 1
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                    tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except (IndexError, KeyError):
            word = w["text"].lower()
            tagged.append(word) 

    return ' '.join(tagged)

In [24]:
# take text input for mystem

all_sents = ''

with open('./all_sents.txt', 'r') as f:
    line = f.readlines()
    for sent in all_text:
        all_sents += sent

In [25]:
#all sents are lemmatized with postags like this

with open('./all_lem_sents1.txt', 'w') as f:
        tagged_lemmas = tag_mystem(all_sents)
        f.write(tagged_lemmas)

### Make a model of word2vec vectors

In [26]:
import gensim
import logging

In [185]:
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)
                    
data = gensim.models.word2vec.LineSentence("all_lem_sents1.txt")
gen_model_tagged = gensim.models.Word2Vec(data, size=30, window=10,min_count=2,sg=0)

2019-05-11 20:57:48,201:WARNING:consider setting layer size to a multiple of 4 for greater performance
2019-05-11 20:57:48,230:INFO:collecting all words and their counts
2019-05-11 20:57:48,411:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-11 20:57:48,782:INFO:collected 48496 word types from a corpus of 573022 raw words and 58 sentences
2019-05-11 20:57:48,794:INFO:Loading a fresh vocabulary
2019-05-11 20:57:50,996:INFO:effective_min_count=2 retains 19178 unique words (39% of original 48496, drops 29318)
2019-05-11 20:57:50,997:INFO:effective_min_count=2 leaves 543704 word corpus (94% of original 573022, drops 29318)
2019-05-11 20:57:51,111:INFO:deleting the raw counts dictionary of 48496 items
2019-05-11 20:57:51,116:INFO:sample=0.001 downsamples 31 most-common words
2019-05-11 20:57:51,127:INFO:downsampling leaves estimated 422215 word corpus (77.7% of prior 543704)
2019-05-11 20:57:51,232:INFO:estimated required memory for 19178 words and 30 dimensio

In [186]:
gen_model_tagged.save('CBOW_30_10_new.model')

2019-05-11 20:58:04,214:INFO:saving Word2Vec object under CBOW_30_10_new.model, separately None
2019-05-11 20:58:04,216:INFO:not storing attribute vectors_norm
2019-05-11 20:58:04,229:INFO:not storing attribute cum_table
2019-05-11 20:58:04,395:INFO:saved CBOW_30_10_new.model


### Prepare lemmatised test and train data

In [31]:
# test lemmas made
test_tagged_lemmas = tag_mystem(test_text_j)

In [32]:
test_tagged_lemmas_ = test_tagged_lemmas.split("\n")

In [34]:
del test_tagged_lemmas_[-1]
len(test_tagged_lemmas)

588823

In [35]:
len(test_text_j.split("\n"))

2045

In [37]:
test.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,Изобретение относится к судостроению и касаетс...,0,0,0,0,0,0,0
1,"Эти состояния называют фазами воды, а превраще...",1,14:22,0:13,23:34,81:81,38:78,81:100
2,И должен ни единой долькой Не отступаться от ...,0,0,0,0,0,0,0
3,Он потребовал обеспечить полное осуществление ...,0,0,0,0,0,0,0
4,"По мнению местного пастора Элла Эбанкса, запре...",0,0,0,0,0,0,0


In [270]:
test['text'][2]

' и_SCONJ   должный_ADJ   ни_PART   единый_ADJ   долька_NOUN    не_PART   отступаться_VERB   от_ADP   лицо_NOUN ,   но_SCONJ   быть_VERB   живой_ADJ ,  живой_ADJ   и_SCONJ   только_PART ,   живой_ADJ   и_SCONJ   только_PART   до_ADP   конец_NOUN . '

In [51]:
# let's make a df with lemmas
test_lem = test
test_lem.drop(["text"], axis=1, inplace=True)
test_lem.insert(0, "text", test_tagged_lemmas_, True)
test_lem.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,изобретение_NOUN относиться_VERB к_ADP с...,0,0,0,0,0,0,0
1,этот_DET состояние_NOUN называть_VERB ф...,1,14:22,0:13,23:34,81:81,38:78,81:100
2,и_SCONJ должный_ADJ ни_PART единый_ADJ ...,0,0,0,0,0,0,0
3,он_PRON потребовать_VERB обеспечивать_VER...,0,0,0,0,0,0,0
4,по_ADP мнение_NOUN местный_ADJ пастор_N...,0,0,0,0,0,0,0


In [267]:
test_lem['text'][45]

' широкий_ADJ   и_SCONJ   тяжелый_ADJ   кисть_NOUN   их_DET   длинный_ADJ   рука_NOUN   походить_VERB   на_ADP   лопата_NOUN ,  а_SCONJ   голова_NOUN  –  на_ADP   таран_NOUN . '

In [114]:
test_lemmas = test_lem

In [55]:
# save the new df
test_lem.to_csv(r'test_lemmas.csv')

train

In [323]:
# train lemmas made
tr_tagged_lemmas = tag_mystem(tr_text)

In [324]:
# used "tyxz" instead of "\n" in tag_mystem to avoid an extra sentence
tr_tagged_lemmas_ = tr_tagged_lemmas.split("tyxz")

In [325]:
len(tr_tagged_lemmas_)

16406

In [326]:
len(tr_text.split("tyxz"))

16406

In [327]:
len(train)

16406

In [328]:
train['text'][5]

'Выворачивает наизнанку, запад превращает в юг, а север — в восток, меняет местами добро и зло, велит открыть своё сердце, ничего не требуя взамен.'

In [329]:
# let's make a df with lemmas
train_lem = train
train_lem.drop(["text"], axis=1, inplace=True)
train_lem.insert(0, "text", tr_tagged_lemmas_, True)
train_lem.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,быть_VERB в_ADP прошлый_ADJ четверг_NOUN...,0,0,0,0,0,0,0
1,работа_NOUN с_ADP двухбайтовый_ADJ на...,1,92:99,83:91,103:109,127:127,119:124,127:134
2,заместитель_NOUN генеральный_ADJ секрет...,0,0,0,0,0,0,0
3,продажа_NOUN недвижимость_NOUN из_ADP ...,0,0,0,0,0,0,0
4,"новый_ADJ являться_VERB то_SCONJ , что...",0,0,0,0,0,0,0


In [330]:
train_lem['text'][6]

'   специалист_NOUN   отмечать_VERB  « грамотный_ADJ   подход_NOUN   к_ADP   ландшафт_NOUN »,  но_SCONJ   рекомендовать_VERB   более_ADV   тщательно_ADV   продумывать_VERB   пляжный_ADJ   зона_NOUN ,  где_ADV   быть_VERB   отдыхать_VERB   не_PART   только_PART   турист_NOUN   из_ADP   отель_NOUN ,  но_SCONJ   и_SCONJ   местный_ADJ   жители'

In [331]:
# save the new df
train_lem.to_csv(r'train_lemmas.csv')

dev

In [318]:
# dev lemmas made
dev_tagged_lemmas = tag_mystem(dev_text)

In [319]:
# used "tyxz" instead of "\n" in tag_mystem to avoid an extra sentence
dev_tagged_lemmas_ = dev_tagged_lemmas.split("tyxz")

In [306]:
len(dev_tagged_lemmas_)

4142

In [320]:
dev_tagged_lemmas_[0]

'центральный_ADJ   часть_NOUN   лунный_ADJ   тень_NOUN ,  где_ADV   наблюдаться_VERB   полный_ADJ   фаза_NOUN   затмение_NOUN   вступать_VERB   на_ADP   земля_NOUN   в_ADP   атлантика_NOUN ,  пересекать_VERB   южный_ADJ   часть_NOUN   африканский_ADJ   континент_NOUN   и_SCONJ   крупный_ADJ   остров_NOUN   мадагаскар_NOUN ,  а_SCONJ   завершать_VERB   свой_DET   путь_NOUN   на_ADP   закат_NOUN   в_ADP   индийский_ADJ   океане'

In [321]:
# let's make a df with lemmas
dev_lem = dev
dev_lem.drop(["text"], axis=1, inplace=True)
dev_lem.insert(0, "text", dev_tagged_lemmas_, True)
dev_lem.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,центральный_ADJ часть_NOUN лунный_ADJ те...,0,0,0,0,0,0,0
1,я_PRON превращать_VERB твой_DET сердц...,1,2:10,11:22,23:30,54:54,34:52,54:60
2,в_ADP данный_ADJ пример_NOUN строка_N...,1,24:37,17:23,38:56,68:68,60:67,68:88
3,ассоциация_NOUN намерен_ADJ занимать_VE...,0,0,0,0,0,0,0
4,""" ты_PRON сам_DET весь_DET портить_VER...",0,0,0,0,0,0,0


In [332]:
dev_lem['text'][0]

'центральный_ADJ   часть_NOUN   лунный_ADJ   тень_NOUN ,  где_ADV   наблюдаться_VERB   полный_ADJ   фаза_NOUN   затмение_NOUN   вступать_VERB   на_ADP   земля_NOUN   в_ADP   атлантика_NOUN ,  пересекать_VERB   южный_ADJ   часть_NOUN   африканский_ADJ   континент_NOUN   и_SCONJ   крупный_ADJ   остров_NOUN   мадагаскар_NOUN ,  а_SCONJ   завершать_VERB   свой_DET   путь_NOUN   на_ADP   закат_NOUN   в_ADP   индийский_ADJ   океане'

In [338]:
# make one df for train and dev
train_lemmas = pd.concat([dev_lem, train_lem], ignore_index=True, sort=False)

In [339]:
len(train_lemmas)

20548

In [340]:
train_lemmas.head()

,text,class,cV,cR1,cR2,V,R1,R2
0,центральный_ADJ часть_NOUN лунный_ADJ те...,0,0,0,0,0,0,0
1,я_PRON превращать_VERB твой_DET сердц...,1,2:10,11:22,23:30,54:54,34:52,54:60
2,в_ADP данный_ADJ пример_NOUN строка_N...,1,24:37,17:23,38:56,68:68,60:67,68:88
3,ассоциация_NOUN намерен_ADJ занимать_VE...,0,0,0,0,0,0,0
4,""" ты_PRON сам_DET весь_DET портить_VER...",0,0,0,0,0,0,0


In [341]:
# save the new train df
train_lemmas.to_csv(r'train_dev_lemmas.csv')

### Check the w2v model

In [187]:
gen_model_tagged=gensim.models.Word2Vec.load('CBOW_30_10_new.model')
gen_model_tagged.init_sims(replace=True)
print(gen_model_tagged.wv.similarity('центральный_ADJ', 'часть_NOUN'))

2019-05-11 20:58:20,601:INFO:loading Word2Vec object from CBOW_30_10_new.model
2019-05-11 20:58:20,731:INFO:loading wv recursively from CBOW_30_10_new.model.wv.* with mmap=None
2019-05-11 20:58:20,733:INFO:setting ignored attribute vectors_norm to None
2019-05-11 20:58:20,734:INFO:loading vocabulary recursively from CBOW_30_10_new.model.vocabulary.* with mmap=None
2019-05-11 20:58:20,736:INFO:loading trainables recursively from CBOW_30_10_new.model.trainables.* with mmap=None
2019-05-11 20:58:20,737:INFO:setting ignored attribute cum_table to None
2019-05-11 20:58:20,738:INFO:loaded CBOW_30_10_new.model
2019-05-11 20:58:20,821:INFO:precomputing L2-norms of word weight vectors


0.8693796


In [188]:
print(gen_model_tagged.wv.similarity('в_ADP', 'сердце_NOUN'))

0.34299147


Prepare data for a bi model

In [412]:
from numpy import array #
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
import keras_metrics as km
from numpy import zeros

In [342]:
# first filter out too long sents 
MAX_length = 60 # words

def filter(df, MAX_length):
    docs = []
    labels = []
    text_index = []
    for i in range(len(df)):        
        sent = ' ' + df['text'][i]       
        if len(sent.split()) <= MAX_length:
            docs.append(sent)
            labels.append(df['class'][i])
         #   print(labels)
            text_index.append(i)
    labels = array(labels)
    return docs, labels, text_index

In [343]:
#docs_test
docs_train, labels_train, text_index = filter(train_lemmas, MAX_length)

In [344]:
len(docs_train)

19874

In [345]:
#docs_test
docs_test, labels_test, test_text_index = filter(test_lemmas, MAX_length)

In [346]:
len(test_lemmas['text'])

2045

In [253]:
len(docs_test)

1992

In [347]:
docs_test[2]

'  и_SCONJ   должный_ADJ   ни_PART   единый_ADJ   долька_NOUN    не_PART   отступаться_VERB   от_ADP   лицо_NOUN ,   но_SCONJ   быть_VERB   живой_ADJ ,  живой_ADJ   и_SCONJ   только_PART ,   живой_ADJ   и_SCONJ   только_PART   до_ADP   конец_NOUN . '

In [348]:
docs_train[2]

'    в_ADP   данный_ADJ   пример_NOUN   строка_NOUN   сгруппировывать_VERB   по_ADP   название_NOUN   товар_NOUN ,  а_SCONJ   столбец_NOUN   по_ADP   категория_NOUN   товар_NOUN . '

In [349]:
def get_vector(sent, model):
    matrix = np.zeros((60,30)) # 60 words, 50 dimentions for a word in the model
    for i, lemma in enumerate(sent.split()):
       # print(lemma)
        if lemma in model.wv:
           # print('in model', model.wv[lemma], 'old',  model[lemma])
            vec = model.wv[lemma]
         #   print(vec)
            matrix[i] = vec
            
        else:
            pass
   # print(matrix)
    vector = np.concatenate(matrix)
   # print(vector, len(vector))
    return vector

In [286]:
docs_train[6]

' сущность   изобретение_NOUN :  в_ADP   ранорасширитель_NOUN   вводить_VERB   съемный_ADJ   цилиндрический_ADJ   дренировать_VERB   эластичный_ADJ   оболочка_NOUN ,  а_SCONJ   сам_DET   ранорасширитель_NOUN   выполнять_VERB   в_ADP   вид_NOUN   тороидальный_ADJ   зигзагообразный_ADJ   элемент_NOUN   из_ADP   материал_NOUN   с_ADP   эффект_NOUN   память_NOUN   формы'

In [411]:
# vectors for train

train_vec = []
for i in range(len(docs_train)):
    sent_vec = get_vector(docs_train[i], gen_model_tagged)
    train_vec.append(sent_vec)

In [355]:
train_vec[40]

array([ 0.02435374, -0.16665716,  0.03422824, ...,  0.        ,
        0.        ,  0.        ])

In [350]:
# vectors for test

test_vec = []
for i in range(len(docs_test)):
    sent_vec = get_vector(docs_test[i], gen_model_tagged)
    test_vec.append(sent_vec)

In [352]:
print(test_vec[9])

[ 0.18666001 -0.24182636 -0.03915953 ...  0.          0.
  0.        ]


In [369]:
model = Sequential()
model.add(Dense(300, input_dim=1800))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', km.binary_precision(), km.binary_recall()])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 300)               540300    
_________________________________________________________________
activation_4 (Activation)    (None, 300)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 200)               60200     
_________________________________________________________________
activation_5 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 201       
Total para

In [371]:
x_train = np.stack(train_vec, axis=0) # to make data readable for Keras
x_test = np.stack(test_vec, axis=0)
y_train = np.asarray(labels_train)
y_test = np.asarray(labels_test)

In [372]:
from sklearn.utils import class_weight  # to balance the classes

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

Train the model

In [413]:
results = model.fit(
    x_train, y_train,
    epochs = 10,
    batch_size = 50,
    validation_data = (x_test, y_test),
    class_weight = class_weights
)

Train on 19874 samples, validate on 1992 samples
Epoch 1/10
19874/19874 [==============================] - 9s 450us/step - loss: 0.2814 - acc: 0.8794 - precision: 0.8382 - recall: 0.8060 - val_loss: 0.4276 - val_acc: 0.8128 - val_precision: 0.7187 - val_recall: 0.7284
Epoch 2/10
19874/19874 [==============================] - 8s 409us/step - loss: 0.2629 - acc: 0.8865 - precision: 0.8432 - recall: 0.8243 - val_loss: 0.4574 - val_acc: 0.8052 - val_precision: 0.7032 - val_recall: 0.7284
Epoch 3/10
19874/19874 [==============================] - 8s 408us/step - loss: 0.2481 - acc: 0.8948 - precision: 0.8583 - recall: 0.8327 - val_loss: 0.4731 - val_acc: 0.8163 - val_precision: 0.7197 - val_recall: 0.7433
Epoch 4/10
19874/19874 [==============================] - 8s 412us/step - loss: 0.2298 - acc: 0.9058 - precision: 0.8716 - recall: 0.8524 - val_loss: 0.4960 - val_acc: 0.7977 - val_precision: 0.6836 - val_recall: 0.7418
Epoch 5/10
19874/19874 [==============================] - 9s 432us/step

In [414]:
score = model.evaluate(x_test, y_test)
score[1] * score[2] * 2 / (score[1] + score[2]) #
# получаем такую F-меру

1992/1992 [==============================] - 0s 107us/step


0.7516283683792009

In [415]:
# save the model

from keras.models import load_model

model.save('gap_model.h5')  

#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('gap_model.h5')

# 6. Оценка результатов бинарной классификации

In [416]:
# make a prediction
prediction = model.predict(x_test)
predi_cate = []
for i in prediction:
    if i < 0.5:
        predi_cate.append(0)
    else:
        predi_cate.append(1)

In [417]:
predi_cate[:5]

[0, 1, 0, 0, 0]

Prepare csv-s for agrr_metrics.py

In [418]:
zeros = []
for i in enumerate(docs_test):
    zeros.append('0')

In [419]:
test_orig = pd.DataFrame({'text': docs_test, 'class_': labels_test, 'cV':zeros, 'cR1':zeros, 'cR2':zeros, 'V':zeros, 'R1':zeros, 'R2':zeros})

In [420]:
test_orig.to_csv(r'test_orig.csv', sep='\t')

In [421]:
test_pred = pd.DataFrame({'text': docs_test, 'class_': predi_cate, 'cV':zeros, 'cR1':zeros, 'cR2':zeros, 'V':zeros, 'R1':zeros, 'R2':zeros})

In [422]:
test_pred.to_csv(r'test_pred.csv', sep='\t')

In [423]:
! python3 agrr_metrics.py -b test_orig.csv test_pred.csv

Binary classification quality (f1-score): 0.7129151291512915
Other metrics: 
 Precision: 0.7051094890510949
 Recall: 0.7208955223880597


Now the result is much better than with one-hot embeddings without pos-tagging, or any other symbols but Russian letters, as in the first run. It is predictable, as gapping is definitely connected with pos and punctuation, and sometimes numbers are also helpful, as they can be cR1/2 or R1/2, so the model could learn meaningful features. Also, three layers instaed of 1 make sense, and around 10 epochs are enough to train such a binary model.